In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import keras
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Input, concatenate, Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras import optimizers
from keras.utils import to_categorical
import os
from imblearn.under_sampling import RandomUnderSampler
from keras.layers import Bidirectional

In [ ]:
MAX_TIME = 30
VOCAB_SIZE = 20000
QUES_CLEANING_PATTERN = re.compile("[\s\n\r\t.,:;\-_\'\"?!#&()\/%\[\]\{\}\<\>\\$@\!\*\+\=]")
LSTM_DIM = 256
LSTM_DIMS = [512,256]
NUM_FILTERS = 5
FILTER_LENGTHS = [1,2,3,4,5,8,10,15,20,25]
DROPOUT = 0.4
LEARNING_RATE = 0.005
NUM_EPOCHS = 10
BATCH_SIZE = 2000
NUM_UNDERSAMPLE = 3
RUS_RATIO = 1.0/4.0

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [ ]:
train_data = np.array(train_data)
test_data = np.array(test_data)

In [ ]:
ct_0 = 0
c = 0
for d in train_data:
    if d[2] == 0:
        ct_0 += 1
    if len(QUES_CLEANING_PATTERN.split(d[1])) > MAX_TIME:
        c+=1
print(ct_0, len(train_data) - ct_0, c/len(train_data))

In [ ]:
#Preprocess
NUM_TRAIN = int(len(train_data)*0.8)
np.random.shuffle(train_data)
train_x = train_data[:NUM_TRAIN,1]
train_y = train_data[:NUM_TRAIN,2]
val_x = train_data[NUM_TRAIN:,1]
val_y = train_data[NUM_TRAIN:,2]
del train_data

In [ ]:
test_x = test_data[:,1]
test_ids = test_data[:,0]
del test_data

In [ ]:
tokenizer = Tokenizer(VOCAB_SIZE)
tokenizer.fit_on_texts(train_x)

In [ ]:
train_x = tokenizer.texts_to_sequences(train_x)
val_x = tokenizer.texts_to_sequences(val_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [ ]:
train_y.shape

In [ ]:
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with open("../input/embeddings/glove.840B.300d/glove.840B.300d.txt", encoding="utf8") as f:
        for line in f:
            values = line.split(' ')
            word = values[0]
            try:
                embeddingVector = np.asarray(values[1:], dtype='float32')
            except:
                print(values)
                break
            embeddingsIndex[word] = embeddingVector

    print('Found %s word vectors.' % len(embeddingsIndex))

    # Minimum word index of any word is 1.
    embeddingMatrix = np.zeros((len(wordIndex) + 1, 300))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    del embeddingsIndex
    return embeddingMatrix

In [ ]:
wordIndex = tokenizer.word_index
wI = {}
for k,v in wordIndex.items():
    if v < VOCAB_SIZE:
        wI[k] = v
wordIndex = wI

In [ ]:
embeddingMatrix = getEmbeddingMatrix(wordIndex)

In [ ]:
train_x = pad_sequences(train_x,maxlen=MAX_TIME,padding='post')
val_x = pad_sequences(val_x,maxlen=MAX_TIME,padding='post')
test_x = pad_sequences(test_x,maxlen=MAX_TIME,padding='post')

In [ ]:
train_y = to_categorical(train_y)
#train_y = train_y.astype('int')
val_y = to_categorical(val_y)

In [ ]:
train_y.shape

In [ ]:
def buildModel(embeddingMatrix):
    embeddingLayer = Embedding(embeddingMatrix.shape[0],300,weights=[embeddingMatrix],\
                              input_length=MAX_TIME,trainable=False)
    '''
    model = Sequential()
    model.add(embeddingLayer)
    model.add(Bidirectional(LSTM(LSTM_DIM,dropout=DROPOUT)))
    #model.add(LSTM(LSTM_DIMS[0],dropout = DROPOUT,return_sequences=True))
    #model.add(LSTM(LSTM_DIMS[1],dropout=DROPOUT))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(2,activation='sigmoid'))
    '''
    sent_inp = Input(shape=(MAX_TIME,))
    sent = embeddingLayer(sent_inp)
    
    _, h1, c1 = LSTM(LSTM_DIM,dropout=DROPOUT,return_state=True)(sent)
    h1 = Reshape([MAX_TIME,LSTM_DIM])(concatenate([h1]*MAX_TIME,1))
    lstm_inp = concatenate([sent,h1],axis=2)
    _, h2, c2 = LSTM(LSTM_DIM*2,dropout=DROPOUT,return_state=True)(lstm_inp)
    probs = Dense(64,activation='relu')(h2)
    probs = Dense(2,activation='sigmoid')(probs)
    model = Model(inputs=sent_inp,outputs=probs)
    rmsprop = optimizers.rmsprop(lr=LEARNING_RATE)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer=rmsprop,
                 metrics=['acc'])
    return model

In [ ]:
model = buildModel(embeddingMatrix)
#model = buildCNNModel(embeddingMatrix)

In [ ]:
'''
for i in range(1):
    rus = RandomUnderSampler(RUS_RATIO,random_state=i)
    train_x_res, train_y_res = rus.fit_resample(train_x,train_y)
    #train_x_res = np.reshape(train_x_res,(train_x_res.shape[0]*train_x_res.shape[1],-1))
    #train_y_res = np.reshape(train_y_res,(train_y_res.shape[0]*train_y_res.shape[1],-1))
    
    #print(train_y_res.shape)
    train_y_res = to_categorical(train_y_res)
    #print(train_y_res.shape)
    perm = np.random.permutation(len(train_y_res))
    train_x_res = train_x_res[perm]
    train_y_res = train_y_res[perm]
    
    #print(train_x_res.shape,train_y_res.shape)
    #break
    model.fit(train_x_res,train_y_res,epochs=NUM_EPOCHS,batch_size=BATCH_SIZE,verbose=1,validation_data=(val_x,val_y))
'''
model.fit(train_x,train_y,epochs=NUM_EPOCHS,batch_size=BATCH_SIZE,verbose=1,validation_data=(val_x,val_y))

In [ ]:
predictions = model.predict(test_x,batch_size=BATCH_SIZE)
predictions = predictions.argmax(axis=1)

In [ ]:
np.sum(predictions==1)/len(test_x)

In [ ]:
np.save("preds.npy",predictions)

In [ ]:
import csv
with open('submission.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['qid','prediction'])
    for i,idx in enumerate(test_ids):
        writer.writerow([idx,predictions[i]])

In [ ]:
val_pred = model.predict(val_x,batch_size=BATCH_SIZE)
val_pred = val_pred.argmax(axis=1)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as fscore

In [ ]:
fs = fscore(val_y.argmax(axis=1),val_pred)
fs